In [1]:
import sys
sys.path.append("..")
import numpy as np
from project.ship import Ship
from project.tests import Tests
from project.botma import TMA
import time

In [2]:
# Класс для сохранения результатов
tests = Tests('test1')

# Создаем наблюдатель
observer_x, observer_y, observer_course, observer_velocity = 0, 0, 0, 5
observer = Ship('Наблюдатель', observer_x, observer_y, observer_course,
                observer_velocity, verbose=True)

In [3]:
# Моделирование траектории наблюдателя
observer.forward_movement(3 * 60)
observer.change_course(270, 'left', omega=0.5)
observer.forward_movement(5 * 60)
observer.change_course(90, 'right', omega=0.5)
observer.forward_movement(3 * 60)

Наблюдатель движется прямо по курсу 0.0° 180с
Наблюдатель перешёл на курс 270.0° за 180с
Наблюдатель движется прямо по курсу 270.0° 300с
Наблюдатель перешёл на курс 90.0° за 360с
Наблюдатель движется прямо по курсу 90.0° 180с


In [4]:
def target_func(seed=None):
    np.random.seed(seed)
    b = 0
    d = 10
    c = np.random.uniform(0, 180)
    v = np.random.uniform(3, 15)
    return [b, d, c, v]
p0 = [0., 25., 90., 7.]
std_arr = [0., 0.1, 0.2, 0.3, 0.5, 1.]
precisions_1 = ['{0:.1f}', '{0:.3f}', '{0:.3f}', '{0:.2f}', '{0:.1f}', '{0:.3f}', '{0:.3f}', '{0:.2f}', '{0:.1f}', '{0:.3f}', '{0:.3f}', '{0:.2f}',
'{0:.2f}', '{0:.2f}', '{0:.2f}', '{0:.2f}', '{0:.2f}', '{0:.2f}', '{:.0f}', '{0:.4f}', '{0:.0f}', '{0:.0f}']
precisions_2 = ['{0:.1f}', '{0:.3f}', '{0:.1f}', '{0:.2f}', '{0:.1f}', '{0:.3f}', '{0:.1f}', '{0:.2f}', '{0:.2f}', '{0:.2f}', '{0:.0f}', '{0:.4f}']

In [5]:
for std in std_arr:

    tma = TMA(observer, sd=np.radians(std), tau=2)
    tma.print_verbose()
    print('ММП v6')
    start_time = time.perf_counter()
    dict_results = tma.swarm(n=10, target_func=target_func, p0=p0, seeded=True)
    stop_time = time.perf_counter()
    print('Моделирование 10 результатов закончено за t = {:.1f} с'.format(stop_time - start_time))
    df = tests.get_df(dict_results)

    d_df = df.copy()
    for i, j in enumerate(df.columns):
        try: d_df[j] = df[j].apply(lambda x: precisions_1[i].format(np.float64(x)))
        except(TypeError): pass
    display(d_df.drop(['СКО X', 'СКО Y', 'СКО VX', 'СКО VY', 'Кб', 'П0_апр', 'Д0_апр', 'К0_апр', 'V0_апр'], axis=1))

    print('Показатели Монте-Карло:')
    for i in range(4):
        a = df['Точ'].apply(lambda x: x[i])
        print('Вариант {}: Рэф = {:.0f}$, Рэф_мин = {:.0f}'.format(i, 100 * a.mean(), min(a)))
    print('Ка = {:.2f}'.format(df['Ка'].mean()))
    print('Nf = {:.0f}, Nf_max = {:.0f}'.format(df['Nf'].mean(), max(df['Nf'])))
    print('Iter = {:.0f}, Iter_max = {:.0f}'.format(df['Iter'].mean(), max(df['Iter'])))
    print('t = {:.4f}, t_max = {:.4f}'.format(df['t'].mean(), max(df['t'])))

    tests.save_df(df, name='../tests/tests8/ЛМ ' + str(np.degrees(tma.standart_deviation)) + '.xlsx')
    print('_______________________________________________')

СКОп = 0.0, tau = 2
ММП v6
Моделирование 10 результатов закончено за t = 0.3 с


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Ка,Точ,t,Nf,Iter
0,0.0,10.000,98.786,11.58,360.0,10.000,98.786,11.58,0.00,"[1, 1, 1, 1]",0.0040,19,15
1,0.0,10.000,75.064,11.64,0.0,10.000,75.064,11.64,0.00,"[1, 1, 1, 1]",0.0030,12,10
2,0.0,10.000,78.479,3.31,360.0,10.000,78.479,3.31,0.00,"[1, 1, 1, 1]",0.0018,8,7
3,0.0,10.000,99.144,11.50,360.0,10.000,99.144,11.50,0.00,"[1, 1, 1, 1]",0.0054,19,15
4,0.0,10.000,174.065,9.57,0.0,10.000,174.065,9.57,0.00,"[1, 1, 1, 1]",0.0049,24,18
5,0.0,10.000,39.959,13.45,360.0,10.000,39.959,13.45,0.00,"[1, 1, 1, 1]",0.0019,9,8
6,0.0,10.000,160.715,6.98,360.0,10.000,160.715,6.98,0.00,"[1, 1, 1, 1]",0.0049,27,21
7,0.0,10.000,13.735,12.36,360.0,10.000,13.735,12.36,0.00,"[1, 1, 1, 1]",0.0018,8,7
8,0.0,10.000,157.217,14.62,360.0,10.000,157.217,14.62,0.00,"[1, 1, 1, 1]",0.0041,23,17
9,0.0,10.000,1.867,9.02,0.0,10.000,1.867,9.02,0.00,"[1, 1, 1, 1]",0.0015,7,6


Показатели Монте-Карло:
Вариант 0: Рэф = 100$, Рэф_мин = 1
Вариант 1: Рэф = 100$, Рэф_мин = 1
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.00
Nf = 16, Nf_max = 27
Iter = 12, Iter_max = 21
t = 0.0033, t_max = 0.0054
_______________________________________________
СКОп = 0.1, tau = 2
ММП v6
Моделирование 10 результатов закончено за t = 0.3 с


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Ка,Точ,t,Nf,Iter
0,0.0,10.000,98.786,11.58,360.0,9.978,98.689,11.57,0.01,"[1, 1, 1, 1]",0.0041,19,15
1,0.0,10.000,75.064,11.64,360.0,10.054,75.108,11.74,0.01,"[1, 1, 1, 1]",0.0026,12,10
2,0.0,10.000,78.479,3.31,0.0,10.007,78.653,3.31,0.01,"[1, 1, 1, 1]",0.0018,8,7
3,0.0,10.000,99.144,11.50,0.0,9.979,99.210,11.45,0.01,"[1, 1, 1, 1]",0.0035,19,15
4,0.0,10.000,174.065,9.57,360.0,10.004,174.043,9.57,0.01,"[1, 1, 1, 1]",0.0053,24,18
5,0.0,10.000,39.959,13.45,0.0,9.982,39.907,13.43,0.01,"[1, 1, 1, 1]",0.0024,9,8
6,0.0,10.000,160.715,6.98,360.0,9.995,160.695,6.98,0.01,"[1, 1, 1, 1]",0.0050,27,21
7,0.0,10.000,13.735,12.36,360.0,9.964,13.672,12.39,0.01,"[1, 1, 1, 1]",0.0021,8,7
8,0.0,10.000,157.217,14.62,0.0,10.008,157.228,14.63,0.01,"[1, 1, 1, 1]",0.0043,23,17
9,0.0,10.000,1.867,9.02,0.0,10.040,1.871,8.97,0.01,"[1, 1, 1, 1]",0.0017,7,6


Показатели Монте-Карло:
Вариант 0: Рэф = 100$, Рэф_мин = 1
Вариант 1: Рэф = 100$, Рэф_мин = 1
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.01
Nf = 16, Nf_max = 27
Iter = 12, Iter_max = 21
t = 0.0033, t_max = 0.0053
_______________________________________________
СКОп = 0.2, tau = 2
ММП v6
Моделирование 10 результатов закончено за t = 0.3 с


...

Показатели Монте-Карло:
Вариант 0: Рэф = 100$, Рэф_мин = 1
Вариант 1: Рэф = 100$, Рэф_мин = 1
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.09
Nf = 16, Nf_max = 27
Iter = 12, Iter_max = 21
t = 0.0032, t_max = 0.0055
_______________________________________________
СКОп = 0.5, tau = 2
ММП v6
Моделирование 10 результатов закончено за t = 0.3 с


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Ка,Точ,t,Nf,Iter
0,0.0,10.000,98.786,11.58,359.9,9.889,98.299,11.52,0.23,"[1, 1, 1, 1]",0.0043,19,15
1,0.0,10.000,75.064,11.64,359.8,10.278,75.282,12.15,0.24,"[1, 1, 1, 1]",0.0024,12,10
2,0.0,10.000,78.479,3.31,0.1,10.037,79.352,3.28,0.25,"[1, 1, 1, 1]",0.0029,8,7
3,0.0,10.000,99.144,11.50,0.1,9.897,99.477,11.26,0.24,"[1, 1, 1, 1]",0.0036,19,15
4,0.0,10.000,174.065,9.57,359.9,10.021,173.956,9.59,0.23,"[1, 1, 1, 1]",0.0048,24,18
5,0.0,10.000,39.959,13.45,0.0,9.910,39.701,13.35,0.25,"[1, 1, 1, 1]",0.0020,9,8
6,0.0,10.000,160.715,6.98,360.0,9.975,160.617,6.96,0.23,"[1, 1, 1, 1]",0.0055,27,21
7,0.0,10.000,13.735,12.36,360.0,9.819,13.421,12.50,0.23,"[1, 1, 1, 1]",0.0019,8,7
8,0.0,10.000,157.217,14.62,0.1,10.040,157.271,14.68,0.27,"[1, 1, 1, 1]",0.0049,23,17
9,0.0,10.000,1.867,9.02,360.0,10.200,1.886,8.75,0.25,"[1, 1, 1, 1]",0.0016,7,6


Показатели Монте-Карло:
Вариант 0: Рэф = 100$, Рэф_мин = 1
Вариант 1: Рэф = 100$, Рэф_мин = 1
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.24
Nf = 16, Nf_max = 27
Iter = 12, Iter_max = 21
t = 0.0034, t_max = 0.0055
_______________________________________________
СКОп = 1.0, tau = 2
ММП v6
Моделирование 10 результатов закончено за t = 0.3 с


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Ка,Точ,t,Nf,Iter
0,0.0,10.000,98.786,11.58,359.9,9.779,97.811,11.46,0.94,"[1, 1, 1, 1]",0.0039,19,15
1,0.0,10.000,75.064,11.64,359.6,10.573,75.498,12.69,0.97,"[0, 1, 1, 1]",0.0025,12,10
2,0.0,10.000,78.479,3.31,0.1,10.074,80.233,3.25,0.98,"[1, 1, 1, 1]",0.0025,8,7
3,0.0,10.000,99.144,11.50,0.2,9.794,99.812,11.03,0.95,"[1, 1, 1, 1]",0.0036,19,15
4,0.0,10.000,174.065,9.57,359.9,10.043,173.846,9.62,0.92,"[1, 1, 1, 1]",0.0050,24,18
5,0.0,10.000,39.959,13.45,0.1,9.822,39.443,13.24,0.99,"[1, 1, 1, 1]",0.0019,9,8
6,0.0,10.000,160.715,6.98,359.9,9.947,160.525,6.94,0.93,"[1, 1, 1, 1]",0.0050,27,21
7,0.0,10.000,13.735,12.36,360.0,9.639,13.115,12.65,0.91,"[1, 1, 1, 1]",0.0017,8,7
8,0.0,10.000,157.217,14.62,0.1,10.081,157.324,14.74,1.09,"[1, 1, 1, 1]",0.0042,23,17
9,0.0,10.000,1.867,9.02,360.0,10.397,1.910,8.49,0.98,"[0, 1, 1, 1]",0.0015,7,6


Показатели Монте-Карло:
Вариант 0: Рэф = 80$, Рэф_мин = 0
Вариант 1: Рэф = 100$, Рэф_мин = 1
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.97
Nf = 16, Nf_max = 27
Iter = 12, Iter_max = 21
t = 0.0032, t_max = 0.0050
_______________________________________________


In [6]:
for std in std_arr:

    tma = TMA(observer, sd=np.radians(std), tau=2)
    tma.print_verbose()
    algorithm_name = 'Метод N пеленгов'
    print(algorithm_name)
    start_time = time.perf_counter()
    dict_results = tma.swarm(n=10, algorithm_name=algorithm_name, target_func=target_func, p0=p0, seeded=True)
    stop_time = time.perf_counter()
    print('Моделирование 10 результатов закончено за t = {:.1f} с'.format(stop_time - start_time))
    df = tests.get_df(dict_results).dropna(axis=1)

    d_df = df.copy()
    for i, j in enumerate(df.columns):
        try: d_df[j] = df[j].apply(lambda x: precisions_2[i].format(np.float64(x)))
        except(TypeError): pass
    display(d_df.drop(['Кб'], axis=1))

    print('Показатели Монте-Карло:')
    for i in range(4):
        a = df['Точ'].apply(lambda x: x[i])
        print('Вариант {}: Рэф = {:.0f}$, Рэф_мин = {:.0f}'.format(i, 100 * a.mean(), min(a)))
    print('Ка = {:.2f}'.format(df['Ка'].mean()))
    print('t = {:.4f}, t_max = {:.4f}'.format(df['t'].mean(), max(df['t'])))
    
    tests.save_df(df, name='../tests/tests8/N ' + str(np.degrees(tma.standart_deviation)) + '.xlsx')
    print('_______________________________________________________')

СКОп = 0.0, tau = 2
Метод N пеленгов
Моделирование 10 результатов закончено за t = 0.2 с


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Ка,Точ,t
0,0.0,10.000,98.8,11.58,0.0,10.000,98.8,11.58,0.00,"[1, 1, 1, 1]",0.0003
1,0.0,10.000,75.1,11.64,0.0,10.000,75.1,11.64,0.00,"[1, 1, 1, 1]",0.0003
2,0.0,10.000,78.5,3.31,0.0,10.000,78.5,3.31,0.00,"[1, 1, 1, 1]",0.0003
3,0.0,10.000,99.1,11.50,0.0,10.000,99.1,11.50,0.00,"[1, 1, 1, 1]",0.0003
4,0.0,10.000,174.1,9.57,0.0,10.000,174.1,9.57,0.00,"[1, 1, 1, 1]",0.0003
5,0.0,10.000,40.0,13.45,0.0,10.000,40.0,13.45,0.00,"[1, 1, 1, 1]",0.0003
6,0.0,10.000,160.7,6.98,0.0,10.000,160.7,6.98,0.00,"[1, 1, 1, 1]",0.0003
7,0.0,10.000,13.7,12.36,0.0,10.000,13.7,12.36,0.00,"[1, 1, 1, 1]",0.0002
8,0.0,10.000,157.2,14.62,0.0,10.000,157.2,14.62,0.00,"[1, 1, 1, 1]",0.0002
9,0.0,10.000,1.9,9.02,0.0,10.000,1.9,9.02,0.00,"[1, 1, 1, 1]",0.0002


Показатели Монте-Карло:
Вариант 0: Рэф = 100$, Рэф_мин = 1
Вариант 1: Рэф = 100$, Рэф_мин = 1
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.00
t = 0.0003, t_max = 0.0003
_______________________________________________________
СКОп = 0.1, tau = 2
Метод N пеленгов
Моделирование 10 результатов закончено за t = 0.2 с


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Ка,Точ,t
0,0.0,10.000,98.8,11.58,359.8,10.091,98.5,11.83,0.01,"[1, 1, 1, 1]",0.0003
1,0.0,10.000,75.1,11.64,359.8,10.088,75.0,11.90,0.01,"[1, 1, 1, 1]",0.0003
2,0.0,10.000,78.5,3.31,0.0,9.993,79.0,3.27,0.01,"[1, 1, 1, 1]",0.0003
3,0.0,10.000,99.1,11.50,359.8,10.144,99.1,11.81,0.01,"[1, 1, 1, 1]",0.0003
4,0.0,10.000,174.1,9.57,360.0,9.987,174.1,9.55,0.01,"[1, 1, 1, 1]",0.0003
5,0.0,10.000,40.0,13.45,360.0,9.937,39.8,13.44,0.01,"[1, 1, 1, 1]",0.0003
6,0.0,10.000,160.7,6.98,0.0,9.887,161.0,6.87,0.01,"[1, 1, 1, 1]",0.0002
7,0.0,10.000,13.7,12.36,359.8,9.789,13.6,12.71,0.01,"[1, 1, 1, 1]",0.0002
8,0.0,10.000,157.2,14.62,360.0,9.988,157.2,14.60,0.01,"[1, 1, 1, 1]",0.0003
9,0.0,10.000,1.9,9.02,360.0,10.023,1.9,8.96,0.01,"[1, 1, 1, 1]",0.0003


Показатели Монте-Карло:
Вариант 0: Рэф = 100$, Рэф_мин = 1
Вариант 1: Рэф = 100$, Рэф_мин = 1
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.01
t = 0.0003, t_max = 0.0003
_______________________________________________________
СКОп = 0.2, tau = 2
Метод N пеленгов
Моделирование 10 результатов закончено за t = 0.2 с


...

Показатели Монте-Карло:
Вариант 0: Рэф = 60$, Рэф_мин = 0
Вариант 1: Рэф = 100$, Рэф_мин = 1
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.10
t = 0.0003, t_max = 0.0005
_______________________________________________________
СКОп = 0.5, tau = 2
Метод N пеленгов
Моделирование 10 результатов закончено за t = 0.2 с


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Ка,Точ,t
0,0.0,10.000,98.8,11.58,359.1,8.998,96.4,10.60,0.37,"[0, 0, 1, 1]",0.0003
1,0.0,10.000,75.1,11.64,359.2,9.409,73.6,11.34,0.32,"[0, 1, 1, 1]",0.0003
2,0.0,10.000,78.5,3.31,0.2,9.737,80.7,2.96,0.26,"[0, 0, 0, 1]",0.0003
3,0.0,10.000,99.1,11.50,359.1,9.091,97.5,10.56,0.40,"[0, 1, 1, 1]",0.0003
4,0.0,10.000,174.1,9.57,360.0,9.700,174.4,9.19,0.24,"[1, 1, 1, 1]",0.0003
5,0.0,10.000,40.0,13.45,359.8,9.062,38.6,12.30,0.29,"[0, 1, 1, 1]",0.0003
6,0.0,10.000,160.7,6.98,0.2,9.076,162.6,6.03,0.25,"[0, 0, 0, 1]",0.0003
7,0.0,10.000,13.7,12.36,359.2,8.588,12.6,13.09,0.31,"[0, 0, 1, 1]",0.0003
8,0.0,10.000,157.2,14.62,360.0,9.496,157.4,13.74,0.30,"[0, 1, 1, 1]",0.0003
9,0.0,10.000,1.9,9.02,360.0,9.843,1.2,8.30,0.26,"[0, 1, 1, 1]",0.0002


Показатели Монте-Карло:
Вариант 0: Рэф = 10$, Рэф_мин = 0
Вариант 1: Рэф = 60$, Рэф_мин = 0
Вариант 2: Рэф = 80$, Рэф_мин = 0
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.30
t = 0.0003, t_max = 0.0003
_______________________________________________________
СКОп = 1.0, tau = 2
Метод N пеленгов
Моделирование 10 результатов закончено за t = 0.2 с


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Ка,Точ,t
0,0.0,10.000,98.8,11.58,358.2,5.952,89.7,6.27,2.67,"[0, 0, 0, 0]",0.0003
1,0.0,10.000,75.1,11.64,358.4,6.952,68.7,8.03,2.01,"[0, 0, 0, 0]",0.0003
2,0.0,10.000,78.5,3.31,0.4,9.019,82.9,2.33,1.14,"[0, 0, 0, 0]",0.0008
3,0.0,10.000,99.1,11.50,358.2,5.770,90.5,5.79,3.20,"[0, 0, 0, 0]",0.0003
4,0.0,10.000,174.1,9.57,359.9,8.908,175.3,8.20,1.11,"[0, 0, 0, 1]",0.0004
5,0.0,10.000,40.0,13.45,359.6,7.261,35.2,9.55,1.49,"[0, 0, 0, 0]",0.0003
6,0.0,10.000,160.7,6.98,0.3,7.668,166.6,4.60,1.13,"[0, 0, 0, 0]",0.0003
7,0.0,10.000,13.7,12.36,358.3,6.727,10.4,11.86,1.55,"[0, 0, 0, 0]",0.0003
8,0.0,10.000,157.2,14.62,359.9,8.166,157.9,11.40,1.47,"[0, 0, 0, 0]",0.0003
9,0.0,10.000,1.9,9.02,360.0,9.115,358.7,6.80,1.15,"[0, 0, 0, 0]",0.0003


Показатели Монте-Карло:
Вариант 0: Рэф = 0$, Рэф_мин = 0
Вариант 1: Рэф = 0$, Рэф_мин = 0
Вариант 2: Рэф = 0$, Рэф_мин = 0
Вариант 3: Рэф = 10$, Рэф_мин = 0
Ка = 1.69
t = 0.0003, t_max = 0.0008
_______________________________________________________


In [7]:
tma = TMA(observer, sd=np.radians(std), tau=2)
tma.print_verbose()
algorithm_name = 'Метод N пеленгов'
print(algorithm_name)
start_time = time.perf_counter()
dict_results = tma.swarm(n=10, algorithm_name=algorithm_name, target_func=target_func, p0=p0, seeded=True)
stop_time = time.perf_counter()
print('Моделирование 10 результатов закончено за t = {:.1f} с'.format(stop_time - start_time))
df = tests.get_df(dict_results)

СКОп = 1.0, tau = 2
Метод N пеленгов
Моделирование 10 результатов закончено за t = 0.3 с
